# 🚀 Spark Cluster - Configuración de Producción

## 📋 Configuración basada en Spark 3.5.3 - Documentación Oficial

### ⚠️ Requisitos mínimos identificados:
- **Driver**: Mínimo 471MB (450MB + overhead)
- **Executor**: Mínimo 471MB (450MB + overhead) 
- **Workers**: Deben tener suficiente memoria para el overhead del sistema + executors

### 📊 Recursos actuales disponibles:
- **Workers**: 2GB cada uno
- **JupyterLab**: 2GB
- **Overhead sistema**: ~1GB por worker
- **Disponible para Spark**: ~1GB por worker


## 🔧 Configuración 1: Cluster Mode Conservador
### Solo 1 executor total para garantizar funcionamiento


In [ ]:
# 🔧 CONFIGURACIÓN CLUSTER CONSERVADORA - 1 EXECUTOR TOTAL
from pyspark.sql import SparkSession
import time

# Detener cualquier sesión existente
try:
    spark.stop()
    print("🛑 Sesión anterior detenida")
    time.sleep(5)
except:
    print("🔍 No había sesión activa")

print("🔄 Creando sesión Spark CLUSTER MODE - Configuración conservadora...")

spark = SparkSession.builder \
    .appName("EducacionIT-Cluster-Conservative") \
    .master("spark://master:7077") \
    .config("spark.executor.memory", "300m") \
    .config("spark.executor.cores", "1") \
    .config("spark.executor.instances", "1") \
    .config("spark.driver.memory", "1g") \
    .config("spark.driver.cores", "1") \
    .config("spark.dynamicAllocation.enabled", "false") \
    .config("spark.shuffle.service.enabled", "false") \
    .config("spark.sql.adaptive.enabled", "false") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "false") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

print(f"✅ Spark iniciado: {spark.version}")
print(f"🔗 Master: {spark.sparkContext.master}")
print(f"📊 Spark UI: http://localhost:4041")
print(f"🎯 App ID: {spark.sparkContext.applicationId}")

print("\n📊 CONFIGURACIÓN CONSERVADORA:")
print("   ✅ 1 executor con 500MB (cumple mínimo de 471MB)")
print("   ✅ Driver con 1GB (excede mínimo de 471MB)")
print("   ✅ Total memoria usada: 1GB")
print("   ✅ Debería funcionar sin problemas")


In [ ]:
# 🧪 TEST BÁSICO DE FUNCIONAMIENTO
print("🧪 PROBANDO CONECTIVIDAD DEL CLUSTER:")

try:
    # Test 1: Operación RDD simple
    print("\n1️⃣ Test RDD básico:")
    rdd = spark.sparkContext.parallelize([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 2)
    result = rdd.sum()
    print(f"   ✅ Suma distribuida: {result}")
    
    # Test 2: DataFrame
    print("\n2️⃣ Test DataFrame:")
    from pyspark.sql.types import StructType, StructField, IntegerType, StringType
    
    data = [(1, "Alice"), (2, "Bob"), (3, "Charlie"), (4, "Diana")]
    schema = StructType([
        StructField("id", IntegerType(), True),
        StructField("name", StringType(), True)
    ])
    
    df = spark.createDataFrame(data, schema)
    count = df.count()
    print(f"   ✅ Conteo distribuido: {count} filas")
    
    # Test 3: Mostrar datos
    print("\n3️⃣ Datos de muestra:")
    df.show()
    
    print("\n🎉 ¡CLUSTER MODE FUNCIONANDO CORRECTAMENTE!")
    print("💡 El executor está procesando trabajos distribuidos")
    
except Exception as e:
    print(f"\n❌ Error en cluster mode: {e}")
    print("🔧 Revisa los logs de los workers")


In [ ]:
# 🗄️ TEST DE HIVE EN CLUSTER MODE
print("🗄️ PROBANDO CONECTIVIDAD CON HIVE:")

try:
    # Verificar bases de datos
    print("\n1️⃣ Verificando bases de datos:")
    databases = spark.sql("SHOW DATABASES").collect()
    print(f"   ✅ Bases de datos encontradas: {len(databases)}")
    for db in databases:
        print(f"      - {db[0]}")
    
    # Conectar a educacionit
    print("\n2️⃣ Conectando a educacionit:")
    spark.sql("USE educacionit")
    print("   ✅ Conectado a educacionit")
    
    # Listar tablas
    print("\n3️⃣ Listando tablas:")
    tables = spark.sql("SHOW TABLES").collect()
    print(f"   ✅ Tablas encontradas: {len(tables)}")
    for table in tables:
        print(f"      - {table[1]}")
    
    # Test de lectura si hay tablas
    if len(tables) > 0:
        table_name = tables[0][1]
        print(f"\n4️⃣ Leyendo datos de {table_name}:")
        df = spark.sql(f"SELECT * FROM {table_name} LIMIT 3")
        print("   📊 Primeras 3 filas:")
        df.show()
        
        count = spark.sql(f"SELECT COUNT(*) as total FROM {table_name}").collect()[0][0]
        print(f"   ✅ Total de registros: {count}")
    
    print("\n🎉 ¡HIVE + CLUSTER MODE FUNCIONANDO PERFECTAMENTE!")
    print("💡 Puedes ejecutar consultas SQL distribuidas")
    
except Exception as e:
    print(f"\n❌ Error con Hive: {e}")
    print("🔧 Verifica que Hive esté funcionando correctamente")


In [ ]:
# 🛑 MODO LOCAL COMO ALTERNATIVA
# Si el cluster sigue fallando, usemos modo local con conectividad a Hive

from pyspark.sql import SparkSession
import time

# Detener cualquier sesión existente
try:
    spark.stop()
    print("🛑 Sesión anterior detenida")
    time.sleep(5)
except:
    print("🔍 No había sesión activa")

print("🔄 Creando sesión Spark MODO LOCAL con Hive...")

spark = SparkSession.builder \
    .appName("EducacionIT-Local-Hive") \
    .master("local[2]") \
    .config("spark.driver.memory", "1g") \
    .config("spark.driver.cores", "2") \
    .config("spark.sql.warehouse.dir", "/tmp/spark-warehouse") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .enableHiveSupport() \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

print(f"✅ Spark iniciado: {spark.version}")
print(f"🔗 Master: {spark.sparkContext.master}")
print(f"📊 Spark UI: http://localhost:4041")
print(f"🎯 App ID: {spark.sparkContext.applicationId}")

print("\n📊 MODO LOCAL CON HIVE:")
print("   ✅ Driver con 1GB")
print("   ✅ 2 cores locales")
print("   ✅ Conectividad completa a Hive")
print("   ✅ Funciona sin problemas de workers")


In [ ]:
# 🎯 CONFIGURACIÓN ULTRA SIMPLE QUE FUNCIONA
# Recursos liberados - ahora probemos con configuración mínima

from pyspark.sql import SparkSession
import time

# Detener cualquier sesión existente
try:
    spark.stop()
    print("🛑 Sesión anterior detenida")
    time.sleep(5)
except:
    print("🔍 No había sesión activa")

print("🔄 Creando sesión Spark ULTRA SIMPLE...")

# Configuración mínima pero funcional
spark = SparkSession.builder \
    .appName("EducacionIT-Simple") \
    .master("spark://master:7077") \
    .config("spark.executor.memory", "800m") \
    .config("spark.executor.cores", "1") \
    .config("spark.executor.instances", "1") \
    .config("spark.driver.memory", "800m") \
    .config("spark.driver.cores", "1") \
    .config("spark.dynamicAllocation.enabled", "false") \
    .config("spark.shuffle.service.enabled", "false") \
    .config("spark.sql.adaptive.enabled", "false") \
    .config("spark.network.timeout", "300s") \
    .config("spark.executor.heartbeatInterval", "60s") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

print(f"✅ Spark iniciado: {spark.version}")
print(f"🔗 Master: {spark.sparkContext.master}")
print(f"📊 Spark UI: http://localhost:4041")
print(f"🎯 App ID: {spark.sparkContext.applicationId}")

print("\n📊 CONFIGURACIÓN ULTRA SIMPLE:")
print("   ✅ 1 executor con 800MB (muy por encima del mínimo)")
print("   ✅ Driver con 800MB (muy por encima del mínimo)")
print("   ✅ Timeouts aumentados para evitar desconexiones")
print("   ✅ Recursos disponibles: 4GB totales")

# Test inmediato
print("\n🧪 Test rápido:")
try:
    rdd = spark.sparkContext.parallelize([1, 2, 3, 4, 5])
    result = rdd.sum()
    print(f"   ✅ Test exitoso - Suma: {result}")
    print("   🎉 ¡CLUSTER MODE FUNCIONANDO!")
except Exception as e:
    print(f"   ❌ Error: {e}")


In [ ]:
# 🗄️ PRUEBA DE CONECTIVIDAD CON HIVE
# Ahora que el cluster funciona, probemos Hive

print("🔍 Probando conectividad con Hive...")

try:
    # Mostrar bases de datos disponibles
    print("\n📊 Bases de datos disponibles:")
    databases = spark.sql("SHOW DATABASES")
    databases.show()
    
    # Usar la base de datos educacionit
    print("\n🎯 Usando base de datos 'educacionit'...")
    spark.sql("USE educacionit")
    print("✅ Base de datos 'educacionit' seleccionada")
    
    # Mostrar tablas disponibles
    print("\n📋 Tablas disponibles en 'educacionit':")
    tables = spark.sql("SHOW TABLES")
    tables.show()
    
    print("🎉 ¡CONECTIVIDAD CON HIVE EXITOSA!")
    
except Exception as e:
    print(f"❌ Error conectando con Hive: {e}")
    print("💡 Verificando si necesitamos habilitar Hive Support...")


In [ ]:
# 🔧 SPARK CLUSTER CON HIVE SUPPORT EXPLÍCITO
# Si la celda anterior falló, probemos con Hive Support habilitado

from pyspark.sql import SparkSession
import time

print("🔄 Recreando sesión Spark con Hive Support explícito...")

# Detener sesión actual
try:
    spark.stop()
    print("🛑 Sesión anterior detenida")
    time.sleep(5)
except:
    print("🔍 No había sesión activa")

# Crear nueva sesión con Hive Support
spark = SparkSession.builder \
    .appName("EducacionIT-Cluster-Hive") \
    .master("spark://master:7077") \
    .config("spark.executor.memory", "800m") \
    .config("spark.executor.cores", "1") \
    .config("spark.executor.instances", "1") \
    .config("spark.driver.memory", "800m") \
    .config("spark.driver.cores", "1") \
    .config("spark.dynamicAllocation.enabled", "false") \
    .config("spark.shuffle.service.enabled", "false") \
    .config("spark.sql.adaptive.enabled", "false") \
    .config("spark.network.timeout", "300s") \
    .config("spark.executor.heartbeatInterval", "60s") \
    .enableHiveSupport() \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

print(f"✅ Spark con Hive iniciado: {spark.version}")
print(f"🔗 Master: {spark.sparkContext.master}")
print(f"📊 Spark UI: http://localhost:4041")

# Test inmediato con Hive
print("\n🧪 Test de conectividad con Hive:")
try:
    databases = spark.sql("SHOW DATABASES")
    databases.show()
    print("🎉 ¡HIVE FUNCIONANDO EN CLUSTER MODE!")
except Exception as e:
    print(f"❌ Error: {e}")


In [ ]:
# 📊 PRUEBAS AVANZADAS CON DATOS DE EDUCACIONIT
# Consultas distribuidas sobre las tablas de Hive

print("🔍 Probando consultas sobre datos de EducacionIT...")

try:
    # Usar la base de datos educacionit
    spark.sql("USE educacionit")
    print("✅ Usando base de datos 'educacionit'")
    
    # Contar registros en tabla clientes
    print("\n👥 Contando clientes:")
    clientes_count = spark.sql("SELECT COUNT(*) as total_clientes FROM clientes")
    clientes_count.show()
    
    # Mostrar algunas ventas
    print("\n💰 Mostrando primeras 5 ventas:")
    ventas_sample = spark.sql("SELECT * FROM venta LIMIT 5")
    ventas_sample.show()
    
    # Consulta agregada: ventas por canal
    print("\n📈 Ventas totales por canal:")
    ventas_por_canal = spark.sql("""
        SELECT c.descripcion as canal, 
               COUNT(*) as num_ventas,
               SUM(v.precio * v.cantidad) as total_ventas
        FROM venta v 
        JOIN canaldeventa c ON v.idcanal = c.idcanal 
        GROUP BY c.descripcion 
        ORDER BY total_ventas DESC
    """)
    ventas_por_canal.show()
    
    print("🎉 ¡CONSULTAS DISTRIBUIDAS FUNCIONANDO!")
    print("🚀 ¡CLUSTER SPARK + HIVE COMPLETAMENTE OPERATIVO!")
    
except Exception as e:
    print(f"❌ Error en consultas: {e}")
    print("💡 Verificando estructura de tablas...")
    try:
        spark.sql("SHOW TABLES").show()
    except:
        print("❌ No se pudo acceder a las tablas")


In [ ]:
# 🚨 ARREGLO INMEDIATO - CONFIGURACIÓN CORRECTA
# El executor volvió a 300MB, necesitamos restaurar los 800MB que funcionaban

from pyspark.sql import SparkSession
import time

print("🚨 ARREGLANDO CONFIGURACIÓN DE MEMORIA...")
print("❌ Problema: Executor con solo 300MB (necesita mínimo 471MB)")
print("✅ Solución: Restaurar configuración de 800MB que funcionaba")

# DETENER CUALQUIER SESIÓN EXISTENTE
try:
    spark.stop()
    print("🛑 Sesión problemática detenida")
    time.sleep(8)  # Esperar más tiempo para limpieza completa
except:
    print("🔍 No había sesión activa")

print("\n🔧 RECREANDO SPARK CON CONFIGURACIÓN CORRECTA...")

# CONFIGURACIÓN EXACTA QUE FUNCIONABA EN CELDA 6
spark = SparkSession.builder \
    .appName("EducacionIT-Fixed-800MB") \
    .master("spark://master:7077") \
    .config("spark.executor.memory", "800m") \
    .config("spark.executor.cores", "1") \
    .config("spark.executor.instances", "1") \
    .config("spark.driver.memory", "800m") \
    .config("spark.driver.cores", "1") \
    .config("spark.dynamicAllocation.enabled", "false") \
    .config("spark.shuffle.service.enabled", "false") \
    .config("spark.sql.adaptive.enabled", "false") \
    .config("spark.network.timeout", "300s") \
    .config("spark.executor.heartbeatInterval", "60s") \
    .enableHiveSupport() \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

print(f"✅ Spark restaurado: {spark.version}")
print(f"🔗 Master: {spark.sparkContext.master}")
print(f"📊 Spark UI: http://localhost:4041")
print(f"🎯 App ID: {spark.sparkContext.applicationId}")

print("\n📊 CONFIGURACIÓN RESTAURADA:")
print("   ✅ Executor: 800MB (muy por encima del mínimo de 471MB)")
print("   ✅ Driver: 800MB")
print("   ✅ Hive Support habilitado")

# TEST INMEDIATO PARA CONFIRMAR
print("\n🧪 Test de confirmación:")
try:
    rdd = spark.sparkContext.parallelize([1, 2, 3, 4, 5])
    result = rdd.sum()
    print(f"   ✅ RDD Test exitoso - Suma: {result}")
    
    # Test Hive
    databases = spark.sql("SHOW DATABASES")
    print(f"   ✅ Hive Test exitoso - Bases de datos encontradas")
    databases.show()
    
    print("🎉 ¡CONFIGURACIÓN ARREGLADA Y FUNCIONANDO!")
    
except Exception as e:
    print(f"   ❌ Error: {e}")
    print("   🔄 Intentando nuevamente...")


In [ ]:
# 🔍 DIAGNÓSTICO: LOS DATOS EXISTEN EN EL METASTORE
# Confirmamos que educacionit y sus 10 tablas están en PostgreSQL
# El problema es la conectividad Spark <-> Metastore

print("🔍 DIAGNÓSTICO COMPLETO:")
print("✅ Base de datos 'educacionit' EXISTE en metastore PostgreSQL")
print("✅ 10 tablas registradas: ventas, compras, gastos, clientes, etc.")
print("❌ Problema: Spark no se conecta correctamente al metastore")

print("\n🧪 Probando conectividad desde Spark:")

try:
    # Forzar reconexión al metastore
    spark.sql("REFRESH")
    
    # Intentar mostrar bases de datos
    print("\n📊 Bases de datos desde Spark:")
    databases = spark.sql("SHOW DATABASES")
    databases.show()
    
    # Contar cuántas bases de datos ve Spark
    db_count = databases.count()
    print(f"🔢 Spark ve {db_count} base(s) de datos")
    
    if db_count >= 2:
        print("✅ Spark ve ambas bases de datos (default + educacionit)")
        
        # Intentar usar educacionit
        print("\n🎯 Intentando usar educacionit:")
        spark.sql("USE educacionit")
        
        # Mostrar tablas
        print("\n📋 Tablas en educacionit:")
        tables = spark.sql("SHOW TABLES")
        tables.show()
        
        print("🎉 ¡PROBLEMA RESUELTO! Spark conectado correctamente")
        
    else:
        print("❌ Spark solo ve 1 base de datos (default)")
        print("💡 Necesitamos reiniciar la sesión Spark con configuración correcta")
        
except Exception as e:
    print(f"❌ Error en conectividad: {e}")
    print("💡 Necesitamos verificar configuración de hive-site.xml")


In [ ]:
# 🎯 CONFIGURACIÓN EXITOSA COPIADA DE spark_cluster_final
# Esta configuración SÍ FUNCIONA con Hive

from pyspark.sql import SparkSession
import time

print("🔄 Recreando Spark con configuración EXITOSA...")

# Detener sesión actual
try:
    spark.stop()
    print("🛑 Sesión anterior detenida")
    time.sleep(5)
except:
    print("🔍 No había sesión activa")

# CONFIGURACIÓN EXACTA QUE FUNCIONA EN spark_cluster_final
spark = SparkSession.builder \
    .appName("EducacionIT-Cluster-Final-Copy") \
    .master("spark://master:7077") \
    .enableHiveSupport() \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.sql.warehouse.dir", "/user/hive/warehouse") \
    .config("spark.executor.memory", "1g") \
    .config("spark.executor.cores", "1") \
    .config("spark.dynamicAllocation.enabled", "false") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

print("🎉 ¡Sesión Spark creada con configuración exitosa!")
print(f"📊 Spark UI: {spark.sparkContext.uiWebUrl}")
print(f"🗄️ Catálogo: {spark.conf.get('spark.sql.catalogImplementation')}")
print(f"🏷️ Aplicación: {spark.sparkContext.appName}")
print(f"🔗 Master: {spark.sparkContext.master}")

# TEST INMEDIATO
print("\n🧪 Test inmediato con configuración exitosa:")
try:
    databases = spark.sql("SHOW DATABASES")
    print("📋 Bases de datos disponibles:")
    databases.show()
    
    db_count = databases.count()
    if db_count >= 2:
        print("🎉 ¡ÉXITO! Spark ve ambas bases de datos")
        
        spark.sql("USE educacionit")
        tables = spark.sql("SHOW TABLES")
        print("📊 Tablas en educacionit:")
        tables.show()
        
        print("🚀 ¡CONFIGURACIÓN REPLICADA EXITOSAMENTE!")
    else:
        print("❌ Aún hay problemas de conectividad")
        
except Exception as e:
    print(f"❌ Error: {e}")


In [ ]:
# 🔧 RECREAR BASE DE DATOS EDUCACIONIT DESDE SPARK
# Como HiveServer2 tiene problemas, vamos a recrear todo desde Spark

print("🔧 RECREANDO BASE DE DATOS EDUCACIONIT...")

try:
    # Crear la base de datos educacionit
    print("📊 Creando base de datos 'educacionit'...")
    spark.sql("CREATE DATABASE IF NOT EXISTS educacionit")
    print("✅ Base de datos 'educacionit' creada")
    
    # Usar la base de datos
    spark.sql("USE educacionit")
    print("✅ Usando base de datos 'educacionit'")
    
    # Verificar que funciona
    print("\n📋 Verificando bases de datos disponibles:")
    databases = spark.sql("SHOW DATABASES")
    databases.show()
    
    print("\n📋 Verificando tablas en educacionit:")
    tables = spark.sql("SHOW TABLES")
    tables.show()
    
    print("🎉 ¡BASE DE DATOS EDUCACIONIT RECREADA!")
    
except Exception as e:
    print(f"❌ Error recreando base de datos: {e}")
    print("💡 Intentando crear tablas de ejemplo...")
